In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import os
from selenium.webdriver.support.ui import Select
import time
import csv
from csv import DictWriter
from collections import OrderedDict
import xlsxwriter
import dask.dataframe as dd
import numpy as np

column_name = ["SEMIS ID", "SEMIS ID (DC)", "SEMIS ID (M&E)", "Status"]

dict = {}

#data_f = pd.read_csv(r'E:/DGME/allteachers.csv', usecols = columns, low_memory=False)

In [2]:
# col_mm = ["School_SEMIS_Code", "School_Name", "Active_Ind"]
# data_mm = pd.read_excel(r'E:/DGME/work/april 9/MNE/school_export_dataset.xlsx', usecols = col_mm , sheet_name="sheet")
data_mm = pd.read_excel(r'E:/DGME/work/april 9/MNE/school_export_dataset.xlsx' , sheet_name="school_export_dataset")
data_mm = pd.DataFrame(data_mm)
data_mm = data_mm.loc[data_mm['ActiveInd']==1]
#data_mm = data_mm[0:100]

# data_mm['SchoolSemisCode'] = data_mm['SchoolSemisCode'].astype(np.int64)
# data_mm['ActiveInd'] = data_mm['ActiveInd'].astype(np.int64())
# print(data_mm['SchoolSemisCode'].dtype, data_mm['ActiveInd'].dtype)

data_mm.head()

,SchoolId,SchoolName,SchoolSemisCode,SchoolPrefix,SchoolLevel,SchoolLevelName,SchoolGenderTypeId,TotalEmployees,SchoolGender,Longitude,Latitude,ActiveInd,Province,District,Tehsil,UnionCouncil,Cities
0,22259,RICHARD ARIF,925000028,GBPS,1,Primary,0,0,Select,68.6955,24.9484,1.0,NaN,Tando Muhammad Khan,Tando Ghulam Hyde,1-Tando Ghulam Hyder,NaN
1,22261,WADHAL CHANG,925000030,GBPS,1,Primary,0,0,Select,68.6758,24.9299,1.0,NaN,Tando Muhammad Khan,Tando Ghulam Hyde,1-Tando Ghulam Hyder,NaN
2,22374,GHULAM AKBER LEGHARI,925000002,GGPS,1,Primary,0,0,Select,68.8167,24.8341,1.0,NaN,Tando Muhammad Khan,Tando Ghulam Hyde,2-Dando,NaN
4,20704,ALI AKBAR MAHAR,917000061,GBPS,1,Primary,0,1,Select,0,0,1.0,NaN,Shaheed Benazirabad,Daur,9-24-Obhari-Swari,NaN
5,27641,Shah Mohd Mashori,100000516,GBPS,1,Primary,0,0,Select,0,0,1.0,NaN,Naushero Feroze,Naushero Feroz,11-Phull,NaN


In [3]:
data_mm

,SchoolId,SchoolName,SchoolSemisCode,SchoolPrefix,SchoolLevel,SchoolLevelName,SchoolGenderTypeId,TotalEmployees,SchoolGender,Longitude,Latitude,ActiveInd,Province,District,Tehsil,UnionCouncil,Cities
0,22259,RICHARD ARIF,925000028,GBPS,1,Primary,0,0,Select,68.6955,24.9484,1.0,NaN,Tando Muhammad Khan,Tando Ghulam Hyde,1-Tando Ghulam Hyder,NaN
1,22261,WADHAL CHANG,925000030,GBPS,1,Primary,0,0,Select,68.6758,24.9299,1.0,NaN,Tando Muhammad Khan,Tando Ghulam Hyde,1-Tando Ghulam Hyder,NaN
2,22374,GHULAM AKBER LEGHARI,925000002,GGPS,1,Primary,0,0,Select,68.8167,24.8341,1.0,NaN,Tando Muhammad Khan,Tando Ghulam Hyde,2-Dando,NaN
4,20704,ALI AKBAR MAHAR,917000061,GBPS,1,Primary,0,1,Select,0,0,1.0,NaN,Shaheed Benazirabad,Daur,9-24-Obhari-Swari,NaN
5,27641,Shah Mohd Mashori,100000516,GBPS,1,Primary,0,0,Select,0,0,1.0,NaN,Naushero Feroze,Naushero Feroz,11-Phull,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50225,43187,LOUG DALL,424010219,GBPS,1,Primary,3,1,Mixed,68.6999,25.3774,1.0,NaN,Tando Allahyar,Tando Allah Yar,8-Bukera Sharif,NaN
50226,43189,SAHIB KHAN JARWAR,424010230,GBLSS,2,Primary,3,5,Mixed,68.7049,25.3535,1.0,NaN,Tando Allahyar,Tando Allah Yar,8-Bukera Sharif,NaN
50227,44732,GARHI DAKHO,414020257,GBPS,1,Primary,3,4,Mixed,68.7013,28.0367,1.0,NaN,Shikarpur,Khanpur,2-Garhi Dakho,NaN
50228,33663,HAJI MUHAMMAD HASHIM,416040336,GBPS,1,Primary,3,1,Mixed,0,0,1.0,NaN,Naushero Feroze,Naushero Feroz,4-Mithiani,NaN


In [5]:
df_mm = data_mm[1000:2000]
df_mm

,SchoolId,SchoolName,SchoolSemisCode,SchoolPrefix,SchoolLevel,SchoolLevelName,SchoolGenderTypeId,TotalEmployees,SchoolGender,Longitude,Latitude,ActiveInd,Province,District,Tehsil,UnionCouncil,Cities
1075,25473,SOHRAB RAJAR,405050207,GGPS,1,Primary,2,1,Girls,69.1325,25.7684,1.0,NaN,Mirpurkhas,Sindhri,2-Hingorno,NaN
1076,12510,SADIQUE KANHAR,415040042,GGPS,1,Primary,2,4,Girls,68.6059,27.5721,1.0,NaN,Khairpur Mirs,Kingri,7-Noorpur,NaN
1077,12512,FATEH ALI CHANDIO,415040045,GGPS,1,Primary,2,3,Girls,68.5795,27.5362,1.0,NaN,Khairpur Mirs,Kingri,7-Noorpur,NaN
1078,12067,JAKHAR.,427040057,GGPS,1,Primary,2,3,Girls,67.8359,27.5163,1.0,NaN,Kambar at Shahdadkot,Warrah,6-Khandoo,NaN
1079,12069,SHER MUHAMMAD BROHI.,427040075,GGPS,1,Primary,2,7,Girls,67.8301,27.4837,1.0,NaN,Kambar at Shahdadkot,Warrah,6-Khandoo,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,22732,DADHORE DARIS,406020342,GBPS,1,Primary,3,1,Mixed,0,0,1.0,NaN,Tharparkar at Mithi,Diplo,6-Dabhro,NaN
2092,22875,MANJTHIO,406020396,GBPS,1,Primary,3,1,Mixed,0,0,1.0,NaN,Tharparkar at Mithi,Diplo,6-Dabhro,NaN
2093,22882,DOONHAI MAIN,406020431,GBPS,1,Primary,3,2,Mixed,0,0,1.0,NaN,Tharparkar at Mithi,Diplo,6-Dabhro,NaN
2094,22889,GABHAN WAGHANI,406020597,GBPS,1,Primary,3,1,Mixed,0,0,1.0,NaN,Tharparkar at Mithi,Diplo,6-Dabhro,NaN


#### Extract Selective Columns 

In [17]:

url = "http://checker.sindheducation.gov.pk/CheckBiometrics.aspx"

# create a new Chrome session
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(5)


# Search DDL
search_ddl = driver.find_element_by_id("ContentPlaceHolder1_ddlSearchBy")
select = Select(search_ddl)
select.select_by_index(2)
time.sleep(2)
data = True
i = 1
for index, row in df_mm.iterrows():
    for j in range(3):        
        search_text = driver.find_element_by_id("ContentPlaceHolder1_txtSearch")
        search_text.click()
        search_text.clear()
        time.sleep(1)
        search_text.send_keys(row['School_SEMIS_Code'])
        search_text.send_keys(Keys.ENTER)
        time.sleep(1)
        search_button = driver.find_element_by_id("ContentPlaceHolder1_btn_search")
        search_button.click()
        time.sleep(2)
        break 

    try:
        data_set = driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[15]')
        semid_dc =  driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[9]')
        semid_me =  driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[12]')
        semid_sg = semid_dc.get_attribute('innerText')
        semid_mm = semid_me.get_attribute('innerText')
        
        #if (semid_dc == semid_me ):
            
        dict[column_name[0]] = row['School_SEMIS_Code']
        dict[column_name[1]] = semid_sg
        dict[column_name[2]] = semid_mm
        dict[column_name[3]] = data_set.get_attribute('innerText')
        print("Record " + str(i) + ": Record Found")
        # data = True
    except:
        dict[column_name[0]] = row['School_SEMIS_Code']
        dict[column_name[3]] = 'Not Found'
        print("Record " + str(i) + ": Not Found")
        # data = False
        # break
    driver.implicitly_wait(3)
    with open(r'D:\SchoolsRecordSet_Status\Status_ActiveSchool_Recordset.csv', 'a', newline='') as f_object:
        dictwriter_object = csv.DictWriter(f_object, fieldnames=column_name)
        if index == 0:
            dictwriter_object.writeheader()
        dictwriter_object.writerow(dict)
        dict.clear()
    i = i + 1;
    
driver.quit();
print("Completed")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Dell\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


Record 1: Record Found
Completed


In [11]:
url = "http://checker.sindheducation.gov.pk/CheckBiometrics.aspx"

# create a new Chrome session
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(5)


# Search DDL
search_ddl = driver.find_element_by_id("ContentPlaceHolder1_ddlSearchBy")
select = Select(search_ddl)
select.select_by_index(2)
time.sleep(2)
data = True
i = 1
for index, row in df_mm.iterrows():
    for j in range(3):       
        search_text = driver.find_element_by_id("ContentPlaceHolder1_txtSearch")
        search_text.click()
        search_text.clear()
        time.sleep(1)
        search_text.send_keys(row['SchoolSemisCode'])
        search_text.send_keys(Keys.ENTER)
        time.sleep(2)
        print(search_text.get_attribute('value'))
        if (search_text.get_attribute('value') != ""):
            search_button = driver.find_element_by_id("ContentPlaceHolder1_btn_search")
            search_button.click()
            time.sleep(4)
            break
        else:
            print("Search Text Can't be Empty")
            continue    

        try:
            data_set = driver.find_element_by_xpath(
                '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[15]')
            semid_dc =  driver.find_element_by_xpath(
                '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[9]')
            semid_me =  driver.find_element_by_xpath(
                    '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[12]')
            semid_sg = semid_dc.get_attribute('innerText')
            semid_mm = semid_me.get_attribute('innerText')

            #if (semid_dc == semid_me ):

            dict[column_name[0]] = row['SchoolSemisCode']
            dict[column_name[1]] = semid_sg
            dict[column_name[2]] = semid_mm
            dict[column_name[3]] = data_set.get_attribute('innerText')
            print("Record " + str(i) + ": Record Found")
                # data = True
        except:
            dict[column_name[0]] = row['School_SEMIS_Code']
            dict[column_name[3]] = 'Not Found'
            print("Record " + str(i) + ": Not Found")
            # data = False
            # break
        driver.implicitly_wait(3)
        with open(r'D:\SchoolsRecordSet_Status\Status_ActiveSchool_Recordset.csv', 'a', newline='') as f_object:
            dictwriter_object = csv.DictWriter(f_object, fieldnames=column_name)
            if index == 0:
                dictwriter_object.writeheader()
            dictwriter_object.writerow(dict)
            dict.clear()
        i = i + 1;
    

            
driver.quit();
print("Completed")            

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Dell\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


415080271
Record 1: Record Found
415080271
Record 2: Record Found
415080271
Record 3: Record Found
415080272
Record 4: Record Found
415080272
Record 5: Record Found
415080272
Record 6: Record Found
415080273
Record 7: Record Found
415080273
Record 8: Record Found
415080273
Record 9: Record Found
415080274
Record 10: Record Found
415080274
Record 11: Record Found
415080274
Record 12: Record Found
415080275
Record 13: Record Found
415080275
Record 14: Record Found
415080275
Record 15: Record Found
415080276
Record 16: Record Found
415080276
Record 17: Record Found
415080276
Record 18: Record Found
415080277
Record 19: Record Found
415080277
Record 20: Record Found
415080277
Record 21: Record Found
415080278
Record 22: Record Found
415080278
Record 23: Record Found
415080278
Record 24: Record Found
415080279
Record 25: Record Found
415080279
Record 26: Record Found
415080279
Record 27: Record Found
415080281
Record 28: Record Found
415080281
Record 29: Record Found
415080281
Record 30: Re

Record 240: Not Found
416010082
Record 241: Record Found
416010082
Record 242: Record Found
416010082
Record 243: Record Found
416010083
Record 244: Record Found
416010083
Record 245: Record Found
416010083
Record 246: Record Found
416010084
Record 247: Record Found
416010084
Record 248: Record Found
416010084
Record 249: Record Found
416010085
Record 250: Record Found
416010085
Record 251: Record Found
416010085
Record 252: Record Found
416010088
Record 253: Record Found
416010088
Record 254: Record Found
416010088
Record 255: Record Found
416010089
Record 256: Not Found
416010089
Record 257: Not Found
416010089
Record 258: Not Found
416010090
Record 259: Record Found
416010090
Record 260: Record Found
416010090
Record 261: Record Found
416010091
Record 262: Record Found
416010091
Record 263: Record Found
416010091
Record 264: Record Found
416010092
Record 265: Record Found
416010092
Record 266: Record Found
416010092
Record 267: Record Found
416010093
Record 268: Record Found
4160100

Record 479: Record Found
416010169
Record 480: Record Found
416010170
Record 481: Record Found
416010170
Record 482: Record Found
416010170
Record 483: Record Found
416010171
Record 484: Record Found
416010171
Record 485: Record Found
416010171
Record 486: Record Found
416010172
Record 487: Record Found
416010172
Record 488: Record Found
416010172
Record 489: Record Found
416010173
Record 490: Record Found
416010173
Record 491: Record Found
416010173
Record 492: Record Found
416010174
Record 493: Record Found
416010174
Record 494: Record Found
416010174
Record 495: Record Found
416010175
Record 496: Record Found
416010175
Record 497: Record Found
416010175
Record 498: Record Found
416010176
Record 499: Record Found
416010176
Record 500: Record Found
416010176
Record 501: Record Found
416010177
Record 502: Record Found
416010177
Record 503: Record Found
416010177
Record 504: Record Found
416010178
Record 505: Record Found
416010178
Record 506: Record Found
416010178
Record 507: Record F

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=89.0.4389.90)


#### Extract Schools Info with Employess Count using SEMIS-ID

In [6]:
column_name = ["SEMIS ID", "Inst. Name (DC)", "SEMIS ID (DC)", "SEMIS ID (M&E)", "Emp. Count", "Status"]
url = "http://checker.sindheducation.gov.pk/CheckBiometrics.aspx"

# create a new Chrome session
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(5)


# Search DDL
search_ddl = driver.find_element_by_id("ContentPlaceHolder1_ddlSearchBy")
select = Select(search_ddl)
select.select_by_index(2)
time.sleep(4)
data = True
i = 1
for index, row in df_mm.iterrows():
    for j in range(3):        
        search_text = driver.find_element_by_id("ContentPlaceHolder1_txtSearch")
        search_text.click()
        search_text.clear()
        search_text.send_keys(row['SchoolSemisCode'])
        search_text.send_keys(Keys.ENTER)
        time.sleep(1)
        search_button = driver.find_element_by_id("ContentPlaceHolder1_btn_search")
        search_button.click()
        time.sleep(6)
        row_count = len(driver.find_elements_by_xpath("//*[@id='ContentPlaceHolder1_gvDCDetails']/tbody/tr"))
        row_count = row_count - 1
        print("Employee_Count: ", row_count)
        break 

    try:
        data_set = driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[15]')
        semid_dc =  driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[9]')
        semid_me =  driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[12]')
        sch_name =  driver.find_element_by_xpath(
            '//*[@id="ContentPlaceHolder1_gvDCDetails"]/tbody/tr[2]/td[10]') 
        semid_sg = semid_dc.get_attribute('innerText')
        semid_mm = semid_me.get_attribute('innerText')
        school = sch_name.get_attribute('innerText')
        #if (semid_dc == semid_me ):
    
        dict[column_name[0]] = row['SchoolSemisCode']
        dict[column_name[1]] = semid_sg
        dict[column_name[2]] = semid_mm
        dict[column_name[3]] = school
        dict[column_name[5]] = data_set.get_attribute('innerText')
        print("Record " + str(i) + ": Record Found")
        # data = True
        if(row_count < 1):
            dict[column_name[4]] = 0
        else:
            dict[column_name[4]] = row_count
            
        
    except:
        dict[column_name[0]] = row['SchoolSemisCode']
        dict[column_name[4]] = 0
        dict[column_name[5]] = 'Not Found'
        print("Record " + str(i) + ": Not Found")
        # data = False
        # break
    driver.implicitly_wait(3)
    with open(r'D:\SchoolsRecordSet_Status\SELD_ActiveSchools_Recordset_Checkers.csv', 'a', newline='') as f_object:
        dictwriter_object = csv.DictWriter(f_object, fieldnames=column_name)
        if index == 0:
            dictwriter_object.writeheader()
        dictwriter_object.writerow(dict)
        dict.clear()
    i = i + 1;
    
driver.quit();
print("Completed")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Dell\.wdm\drivers\chromedriver\win32\90.0.4430.24]


Employee_Count:  1
Record 1: Record Found
Employee_Count:  4
Record 2: Record Found
Employee_Count:  3
Record 3: Record Found
Employee_Count:  3
Record 4: Record Found
Employee_Count:  7
Record 5: Record Found
Employee_Count:  4
Record 6: Record Found
Employee_Count:  13
Record 7: Record Found
Employee_Count:  8
Record 8: Record Found
Employee_Count:  2
Record 9: Record Found
Employee_Count:  -1
Record 10: Not Found
Employee_Count:  -1
Record 11: Not Found
Employee_Count:  7
Record 12: Record Found
Employee_Count:  1
Record 13: Record Found
Employee_Count:  1
Record 14: Record Found
Employee_Count:  1
Record 15: Record Found
Employee_Count:  1
Record 16: Record Found
Employee_Count:  3
Record 17: Record Found
Employee_Count:  3
Record 18: Record Found
Employee_Count:  -1
Record 19: Not Found
Employee_Count:  35
Record 20: Record Found
Employee_Count:  1
Record 21: Record Found
Employee_Count:  2
Record 22: Record Found
Employee_Count:  8
Record 23: Record Found
Employee_Count:  2
Recor

Employee_Count:  1
Record 192: Record Found
Employee_Count:  2
Record 193: Record Found
Employee_Count:  4
Record 194: Record Found
Employee_Count:  1
Record 195: Record Found
Employee_Count:  -1
Record 196: Not Found
Employee_Count:  2
Record 197: Record Found
Employee_Count:  1
Record 198: Record Found
Employee_Count:  2
Record 199: Record Found
Employee_Count:  3
Record 200: Record Found
Employee_Count:  2
Record 201: Record Found
Employee_Count:  11
Record 202: Record Found
Employee_Count:  27
Record 203: Record Found
Employee_Count:  1
Record 204: Record Found
Employee_Count:  1
Record 205: Record Found
Employee_Count:  -1
Record 206: Not Found
Employee_Count:  -1
Record 207: Not Found
Employee_Count:  2
Record 208: Record Found
Employee_Count:  1
Record 209: Record Found
Employee_Count:  -1
Record 210: Not Found
Employee_Count:  -1
Record 211: Not Found
Employee_Count:  1
Record 212: Record Found
Employee_Count:  1
Record 213: Record Found
Employee_Count:  1
Record 214: Record Fo

Employee_Count:  1
Record 380: Record Found
Employee_Count:  6
Record 381: Record Found
Employee_Count:  1
Record 382: Record Found
Employee_Count:  1
Record 383: Record Found
Employee_Count:  -1
Record 384: Not Found
Employee_Count:  23
Record 385: Record Found
Employee_Count:  -1
Record 386: Not Found
Employee_Count:  -1
Record 387: Not Found
Employee_Count:  1
Record 388: Record Found
Employee_Count:  1
Record 389: Record Found
Employee_Count:  1
Record 390: Record Found
Employee_Count:  6
Record 391: Record Found
Employee_Count:  1
Record 392: Record Found
Employee_Count:  3
Record 393: Record Found
Employee_Count:  1
Record 394: Record Found
Employee_Count:  1
Record 395: Record Found
Employee_Count:  4
Record 396: Record Found
Employee_Count:  70
Record 397: Record Found
Employee_Count:  3
Record 398: Record Found
Employee_Count:  25
Record 399: Record Found
Employee_Count:  2
Record 400: Record Found
Employee_Count:  3
Record 401: Record Found
Employee_Count:  1
Record 402: Reco

Employee_Count:  2
Record 568: Record Found
Employee_Count:  1
Record 569: Record Found
Employee_Count:  1
Record 570: Record Found
Employee_Count:  3
Record 571: Record Found
Employee_Count:  -1
Record 572: Not Found
Employee_Count:  2
Record 573: Record Found
Employee_Count:  1
Record 574: Record Found
Employee_Count:  7
Record 575: Record Found
Employee_Count:  1
Record 576: Record Found
Employee_Count:  1
Record 577: Record Found
Employee_Count:  4
Record 578: Record Found
Employee_Count:  3
Record 579: Record Found
Employee_Count:  -1
Record 580: Not Found
Employee_Count:  1
Record 581: Record Found
Employee_Count:  1
Record 582: Record Found
Employee_Count:  1
Record 583: Record Found
Employee_Count:  1
Record 584: Record Found
Employee_Count:  2
Record 585: Record Found
Employee_Count:  -1
Record 586: Not Found
Employee_Count:  -1
Record 587: Not Found
Employee_Count:  -1
Record 588: Not Found
Employee_Count:  1
Record 589: Record Found
Employee_Count:  -1
Record 590: Not Found


Employee_Count:  2
Record 756: Record Found
Employee_Count:  -1
Record 757: Not Found
Employee_Count:  3
Record 758: Record Found
Employee_Count:  2
Record 759: Record Found
Employee_Count:  2
Record 760: Record Found
Employee_Count:  3
Record 761: Record Found
Employee_Count:  -1
Record 762: Not Found
Employee_Count:  -1
Record 763: Not Found
Employee_Count:  1
Record 764: Record Found
Employee_Count:  1
Record 765: Record Found
Employee_Count:  21
Record 766: Record Found
Employee_Count:  1
Record 767: Record Found
Employee_Count:  2
Record 768: Record Found
Employee_Count:  2
Record 769: Record Found
Employee_Count:  1
Record 770: Record Found
Employee_Count:  -1
Record 771: Not Found
Employee_Count:  1
Record 772: Record Found
Employee_Count:  2
Record 773: Record Found
Employee_Count:  1
Record 774: Record Found
Employee_Count:  -1
Record 775: Not Found
Employee_Count:  1
Record 776: Record Found
Employee_Count:  3
Record 777: Record Found
Employee_Count:  1
Record 778: Record Fou

Employee_Count:  1
Record 944: Record Found
Employee_Count:  1
Record 945: Record Found
Employee_Count:  1
Record 946: Record Found
Employee_Count:  1
Record 947: Record Found
Employee_Count:  -1
Record 948: Not Found
Employee_Count:  1
Record 949: Record Found
Employee_Count:  -1
Record 950: Not Found
Employee_Count:  1
Record 951: Record Found
Employee_Count:  3
Record 952: Record Found
Employee_Count:  -1
Record 953: Not Found
Employee_Count:  -1
Record 954: Not Found
Employee_Count:  1
Record 955: Record Found
Employee_Count:  1
Record 956: Record Found
Employee_Count:  2
Record 957: Record Found
Employee_Count:  1
Record 958: Record Found
Employee_Count:  1
Record 959: Record Found
Employee_Count:  -1
Record 960: Not Found
Employee_Count:  -1
Record 961: Not Found
Employee_Count:  4
Record 962: Record Found
Employee_Count:  5
Record 963: Record Found
Employee_Count:  1
Record 964: Record Found
Employee_Count:  10
Record 965: Record Found
Employee_Count:  -1
Record 966: Not Found
E